# Fun with Floating Point Precision in numpy

I recently had a bug in my code that obviously was caused by an issue with floating point precision but had me scratching my head how it came about. Here it is:

In [1]:
import numpy as np
from astropy.table import Table

`Table` lets me read a FITS table, the standard data format in Astronomy and lets me access table columns as `numpy.ndarray`.

In [2]:
tab = Table.read("data/float.fits")
x = tab['x']
type(x)

FileNotFoundError: [Errno 2] No such file or directory: 'data/float.fits'